In [204]:
%reload_ext autoreload
%autoreload 2

from data_inputs import *
from sim_data import *
from models import *
import settings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Static Model Optimization

In [205]:
## TODO: modular inputs: batt size, gt startup cost, gt fuel cost, demand min

# Static, non-stochastic
static_df = get_static_vpp_output()
static_df.head()

,hour,batt_flow,batt_energy,turb_on,turb_start,turb_gen,solar_gen,wind_gen,net_gen,net_out,elect_price,hourly_profit
0,0,40.00,120.00,0,0,0,0.0,6.55,6.55,46.55,32.48,1511.944
1,1,3.24,123.04,0,0,0,0.0,8.76,8.76,12.00,32.40,388.800
2,2,3.24,126.08,0,0,0,0.0,8.76,8.76,12.00,32.14,385.680
3,3,6.19,131.89,0,0,0,0.0,5.81,5.81,12.00,32.08,384.960
4,4,8.40,139.77,0,0,0,0.0,3.60,3.60,12.00,31.18,374.160


In [206]:
weekly_profit_static = sum(static_df.hourly_profit)
weekly_profit_static

120472.61836666673

### Distribution of Stochastic Profits from Static model

In [ ]:
p_sim, w_sim, l_sim, sim_perm = get_sim_data()
p,w,l = get_input_data()
rev_check = sum(l[t]*(static_df['batt_flow'][t] + p[t] + w[t] + settings.G*static_df['turb_on'][t]) -\
        settings.CG*static_df['turb_on'][t] - settings.S*static_df['turb_start'][t] for t in range(settings.T))
print("rev check is: ", rev_check)

rev_static = []
net_gen_static = []
for idx, sim in enumerate(sim_perm):
    p = p_sim['sim_%d'%sim[0]]
    w = w_sim['sim_%d'%sim[1]]
    l = l_sim['sim_%d'%sim[2]]
    net_gen_static.append([0]*settings.T)
    min_gen_check = {}
    for t in range(settings.T):
        net_gen_static[idx][t] = static_df['batt_flow'][t] + p[t] + w[t] + settings.G*static_df['turb_on'][t]
        min_gen_check[t] = (1 if net_gen_static[idx][t] < settings.MIN_GEN else 0)
    rev_static.append(sum(l[t]*(static_df['batt_flow'][t] + p[t] + w[t] + settings.G*static_df['turb_on'][t]) -\
            (settings.CG*static_df['turb_on'][t]) - settings.S*static_df['turb_start'][t] -\
            settings.FINE*min_gen_check[t] for t in range(settings.T)))



    
exp_rev_static = int(sum(rev_static) / len(rev_static))
print("Expected Revenue for Static Secnario is:", exp_rev_static)
fig, ax = plt.subplots(tight_layout=True)
hist = ax.hist(rev_static)
hist = ax.set_title('Distribution of Net Weekly Revenue for Static Scenarios')

In [ ]:
min_gen = [settings.MIN_GEN]*settings.T

%matplotlib inline
plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
for idx in range(len(net_gen_static)):
    plot_stat = plt.plot(net_gen_static[idx], label=idx)
plot_stat = plt.plot(min_gen, label = "Minimum Allowed Power Generation")
#plt.legend()
plt.xlabel('Time (h)')
plt.ylabel('Power (MWh)')
plt.title('Static Forecast Optimization')

Plot above shows that for the static optimization run through the static scenarios, min power generation often drops below min gen. We will need to compare static to stochastic without the batter constraint.

In [ ]:
%matplotlib inline
plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
plot_stat = plt.plot(static_df['batt_flow'], label='battery')
plot_stat = plt.plot(static_df['turb_gen'], label='turbine')
plot_stat = plt.plot(static_df['solar_gen'], label='solar')
plot_stat = plt.plot(static_df['wind_gen'], label='wind')
plot_stat = plt.plot(static_df['net_out'], color='m', linewidth=3, label='net output')
plot_stat = plt.plot(static_df['net_gen'], color='k', linewidth=3, label='net generation')
plt.legend()
plt.xlabel('Time (h)')
plt.ylabel('Power (MWh)')
plt.title('Static Forecast Optimization')

### Stochastic Model Optimization

In [ ]:
# Stochastic
stoc_df, rev_stoc = get_stochastic_vpp_output()
stoc_df.head()

In [ ]:
weekly_profit_stoc = sum(rev_stoc)/(settings.SIMS**3)
weekly_profit_stoc

In [ ]:
plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
plot_stat = plt.plot(stoc_df['batt_flow'], label='battery')
plot_stat = plt.plot(stoc_df['turb_gen'], label='turbine')
plot_stat = plt.plot(stoc_df['solar_gen'], label='solar')
plot_stat = plt.plot(stoc_df['wind_gen'], label='wind')
plot_stat = plt.plot(stoc_df['net_out'], color='m', linewidth=3, label='net output')
plot_stat = plt.plot(stoc_df['net_gen'], color='k', linewidth=3, label='net generation')
plt.legend()
plt.xlabel('Time (h)')
plt.ylabel('Power (MWh)')
plt.title('Stochastic Forecast Optimization')

In [ ]:
fig, ax = plt.subplots(tight_layout=True)
hist = ax.hist(rev_stoc)
hist = ax.set_title('Distribution of Net Weekly Revenue for Stochastic Scenarios')

### No Battery Static

In [ ]:
nb_static_df = get_static_nb_output()
nb_static_df.head()

In [ ]:
weekly_profit_nb = sum(nb_static_df.hourly_profit)
weekly_profit_nb

In [ ]:
plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
plot_nb = plt.plot(nb_static_df['turb_gen'], label='turbine')
plot_nb = plt.plot(nb_static_df['solar_gen'], label='solar')
plot_nb = plt.plot(nb_static_df['wind_gen'], label='wind')
plot_nb = plt.plot(nb_static_df['net_out'], color='m', linewidth=3, label='net output')
plot_nb = plt.plot(nb_static_df['net_gen'], color='k', linewidth=3, label='net generation')
plt.legend()
plt.xlabel('Time (h)')
plt.ylabel('Power (MWh)')
plt.title('No Battery, Static Forecast Optimization')

### No Battery, Stochastic

In [ ]:
nb_stoc_df = get_stochastic_nb_output()
nb_stoc_df.head()

In [ ]:
model,g,s,revenue = optimize_vpp_stochastic_no_battery()

In [ ]:
weekly_profit_nb_stoc = sum(nb_stoc_df.hourly_profit)
weekly_profit_nb_stoc

In [ ]:
plt.figure(figsize=(18, 8), dpi= 80, facecolor='w', edgecolor='k')
plot_nb = plt.plot(nb_stoc_df['turb_gen'], label='turbine')
plot_nb = plt.plot(nb_stoc_df['solar_gen'], label='solar')
plot_nb = plt.plot(nb_stoc_df['wind_gen'], label='wind')
plot_nb = plt.plot(nb_stoc_df['net_out'], color='m', linewidth=3, label='net output')
plot_nb = plt.plot(nb_stoc_df['net_gen'], color='k', linewidth=3, label='net generation')
plt.legend()
plt.xlabel('Time (h)')
plt.ylabel('Power (MWh)')
plt.title('No Battery, Stochastic Forecast Optimization')

In [ ]:
p_sim, w_sim, l_sim, sim_perm = get_sim_data()

In [ ]:
p_sim.set_index('hour', inplace=True)
w_sim.set_index('hour', inplace=True)
l_sim.set_index('hour', inplace=True)

In [ ]:
styles = ['k-','r-.','y-.', 'b-.', 'c-.', 'm-.', 'g-.',\
          'r--','y--', 'b--', 'c--', 'm--', 'g--']
styles = styles[:settings.SIMS+1]
linewidths = [3] + [1]*settings.SIMS

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
fig.suptitle('Simulation Data')

for col, style, lw in zip(p_sim.columns, styles, linewidths):
    p_sim[col].plot(style=style, lw=lw, ax=ax1)
ax1.set(ylabel='Solar Power (MWh)')
ax1.yaxis.grid(color='gray', linestyle='solid')

for col, style, lw in zip(w_sim.columns, styles, linewidths):
    w_sim[col].plot(style=style, lw=lw, ax=ax2)
ax2.set(ylabel='Wind Power (MWh)')
ax2.yaxis.grid(color='gray', linestyle='solid')

for col, style, lw in zip(l_sim.columns, styles, linewidths):
    l_sim[col].plot(style=style, lw=lw, ax=ax3)
ax3.set(ylabel='Electricity Price ($)')
ax3.yaxis.grid(color='gray', linestyle='solid')

ax1.legend(p_sim.columns[1:].insert(0,'actual forecast'),\
           bbox_to_anchor=(1.04,1), loc='upper left')
plt.subplots_adjust(right=0.7)
fig.show()


### Cost Analysis
- Battery cost = \$398/kWh  
- System capacity = 352 MWh  
- Revenue difference ~ $100k/wk  

In [ ]:
batt_cap = 352*1000  # kwh
batt_cost = batt_cap * 200
batt_cost

In [ ]:
payback_weeks = batt_cost / 10e4
payback_weeks

In [ ]:
payback_yrs = payback_weeks/52
payback_yrs

In [ ]:
2600/6